<a href="https://colab.research.google.com/github/EderLara/CuadernosPythonParaML/blob/main/Tutorial_Polars_Elemental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **POLARS - Introducción a Polars**

## **¿Qué es Polars?**

Polars es una biblioteca de manipulación de DataFrames escrita en Rust (un lenguaje de programación conocido por su seguridad y velocidad) y que utiliza Apache Arrow como su modelo de memoria columnar en segundo plano.

Está diseñada desde cero para el procesamiento en paralelo y la optimización de consultas, lo que la hace significativamente más rápida que Pandas en muchas operaciones, especialmente con datasets grandes.

## **¿Por qué Polars? (Ventajas y Cuándo Considerarlo)**

Si bien Pandas es fantástico, Polars brilla en ciertos escenarios:

1. **Rendimiento Superior:**
    * **Velocidad:** Al estar construido en Rust y diseñado para el paralelismo (aprovecha múltiples núcleos de tu CPU automáticamente), Polars puede ser órdenes de magnitud más rápido que Pandas para muchas operaciones (filtrado, agrupaciones, joins).
    * **Manejo Eficiente de Memoria:** Utiliza Apache Arrow, que es un formato columnar eficiente, y tiene estrategias para minimizar el uso de memoria y las copias de datos.
2. **API Expresiva y Moderna:**
    * **Encadenamiento de Métodos (Method Chaining):** La sintaxis de Polars se presta muy bien a encadenar operaciones de forma clara y legible.
    * **"Expression API" (API de Expresiones): **Este es uno de los conceptos más poderosos de Polars. Te permite definir operaciones complejas sobre columnas de una manera muy flexible y optimizable. Veremos esto en detalle.
    * **Lazy Evaluation (Evaluación Perezosa):** Por defecto, muchas operaciones en Polars se construyen como un "plan de consulta" (modo lazy) y solo se ejecutan cuando es estrictamente necesario (por ejemplo, al llamar a .collect()). Esto permite a Polars optimizar toda la cadena de operaciones antes de la ejecución.

3. **Soporte para Out-of-Core (Streaming):** Polars puede procesar datasets que son más grandes que la memoria RAM disponible utilizando su API de scan_* (ej. scan_csv, scan_parquet). En este modo, Polars procesa los datos en trozos (chunks) sin necesidad de cargarlos todos en memoria.
4. **Tipado Estricto y Apache Arrow:** El uso de Arrow asegura una gestión de tipos de datos más robusta y eficiente, además de facilitar la interoperabilidad con otros sistemas que también usan Arrow.

---

## **Similitudes Conceptuales con Pandas:**
Si vienes de Pandas, muchos conceptos te resultarán familiares:

* **DataFrame:** La estructura tabular principal.
* **Series:** Representa una columna individual.
* Operaciones como **selección**, **filtrado**, **agrupación**, **joins**, etc., existen en Polars, aunque la sintaxis para lograrlas puede diferir.

## **Diferencias Clave con Pandas (a Alto Nivel):**

1. **Índices (Indexes):** Polars no tiene un concepto de índice de fila mutable como el de Pandas (ej. `df.loc[]` basado en etiquetas de índice). Las filas se identifican principalmente por su posición entera. Esto simplifica la API en algunos aspectos y la hace más performante, ya que no hay que mantener la sobrecarga de un índice. Las operaciones se centran más en las columnas y sus valores.
2. **Mutabilidad:** Polars favorece fuertemente la inmutabilidad. La mayoría de las operaciones devuelven un nuevo DataFrame o Serie, en lugar de modificar el original "in-place". Esto ayuda a prevenir efectos secundarios inesperados.
3. **Lazy vs. Eager Execution:**
    * **Eager (Ansioso):** Las operaciones se ejecutan inmediatamente (como en Pandas por defecto).
    * **Lazy (Perezoso):** Las operaciones se registran en un plan y solo se ejecutan cuando se llama a `.collect()`. Esto permite a Polars aplicar optimizaciones al plan completo. El modo lazy es el más potente y recomendado para rendimiento.
4. **Sintaxis de Selección y Manipulación:** Aunque los objetivos son los mismos, la forma de seleccionar columnas, filtrar filas y aplicar transformaciones (especialmente con la API de expresiones) es diferente y muy característica de Polars.

---
## **Instalación e Importación**
1. Instalación Básica (instala los componentes comunes):
```
pip install polars
```

2. O, si quieres incluir todas las funcionalidades extra (como conectores a diferentes tipos de archivos, etc.):
```
pip install polars[all]
```

3. Importación (La convención es importar Polars con el alias pl):
```
import polars as pl
```

---
# **Tabla de Contenido**

1. **Introducción:** Qué es, por qué usarlo, diferencias con Pandas.
2. **Creación de Series y DataFrames:** `pl.Series`, `pl.DataFrame`, `pl.from_dicts`.
3. **Inspección Básica:** `head`, `tail`, `shape`, `describe`, `dtypes`, `schema`, `glimpse`.
4. **Selección y Filtrado:** `select`, `filter`, Expresiones `pl.col()`, `pl.selectors`.
5. **Manipulación de Datos:** `with_columns`, `drop`, `fill_null`, `rename`, `cast`, `sort`, `map_elements`.
6. **Agrupación y Agregación:** `group_by`, `agg` con expresiones, `pivot`.
7. **Entrada/Salida (I/O):** `read_csv`/`scan_csv`, `read_excel`, `read_parquet`/`scan_parquet`, y sus contrapartes `write_`*. Interoperabilidad.

# **Creación de Series y DataFrames**

## **Creando Series en Polars (`pl.Series`)**

Una Serie en Polars es similar a la de Pandas: un array unidimensional de datos.

1. **Desde una lista de Python:**

In [ ]:
import polars as pl

# Creando una Serie desde una lista, especificando el nombre
datos_lista_pl = [10, 20, 30, 40, 50]
serie_pl_lista = pl.Series(name="mis_numeros", values=datos_lista_pl)
print("Serie de Polars desde una lista:")
print(serie_pl_lista)
print("\nTipo de dato de la Serie:", serie_pl_lista.dtype)
print("Nombre de la Serie:", serie_pl_lista.name)
print("Longitud de la Serie:", len(serie_pl_lista)) # o serie_pl_lista.len()

"""
# Salida Esperada:

Serie de Polars desde una lista:
shape: (5,)
Series: 'mis_numeros' [i64]
[
    10
    20
    30
    40
    50
]

Tipo de dato de la Serie: Int64
Nombre de la Serie: mis_numeros
Longitud de la Serie: 5

"""

2. **Especificando el tipo de dato (`dtype`):**

Polars tiene su propio sistema de tipos de datos (ej. pl.Int32, pl.Float64, pl.Utf8 para strings, pl.Boolean, pl.Date, pl.Datetime).

In [ ]:
import polars as pl

datos_strings = ["manzana", "banana", "pera"]
# Creando una serie de strings (Utf8 es el tipo para texto en Polars)
serie_strings = pl.Series(name="frutas", values=datos_strings, dtype=pl.Utf8)
print("\nSerie de Polars de strings:")
print(serie_strings)

datos_floats = [1.0, 2.5, 3.0]
serie_floats = pl.Series(name="decimales", values=datos_floats, dtype=pl.Float32) # especificamos Float32
print("\nSerie de Polars de floats (Float32):")
print(serie_floats)

## **Creando DataFrames en Polars (`pl.DataFrame`)**

1. **Desde un diccionario de Python (listas, pl.Series, arrays de NumPy):**

Esta es la forma más común. Las claves del diccionario son los nombres de las columnas y los valores son los datos de esas columnas.

In [ ]:
import polars as pl
import numpy as np

datos_df_pl = {
    'ID_Producto': [101, 102, 103, 104],
    'Nombre_Producto': ['Teclado', 'Mouse', 'Monitor', 'Webcam'],
    'Precio': [75.0, 25.5, 300.0, 45.99],
    'Stock': np.array([50, 120, 30, 75]) # Podemos usar arrays de NumPy
}
df_pl = pl.DataFrame(datos_df_pl)
print("\nDataFrame de Polars desde un diccionario:")
print(df_pl)
print("\nSchema del DataFrame:") # El schema es como el .info() pero más enfocado en tipos
print(df_pl.schema)

"""
DataFrame de Polars desde un diccionario:
shape: (4, 4)
┌─────────────┬─────────────────┬────────┬───────┐
│ ID_Producto ┆ Nombre_Producto ┆ Precio ┆ Stock │
│ ---         ┆ ---             ┆ ---    ┆ ---   │
│ i64         ┆ str             ┆ f64    ┆ i64   │
╞═════════════╪═════════════════╪════════╪═══════╡
│ 101         ┆ Teclado         ┆ 75.0   ┆ 50    │
│ 102         ┆ Mouse           ┆ 25.5   ┆ 120   │
│ 103         ┆ Monitor         ┆ 300.0  ┆ 30    │
│ 104         ┆ Webcam          ┆ 45.99  ┆ 75    │
└─────────────┴─────────────────┴────────┴───────┘

Schema del DataFrame:
OrderedDict([('ID_Producto', Int64), ('Nombre_Producto', Utf8), ('Precio', Float64), ('Stock', Int64)])

Polars también infiere los tipos aquí.
La representación visual del DataFrame en la consola es muy útil.
"""

2. **Definiendo el Schema explícitamente:**

Se pueden definir los tipos de datos de cada columna al crear el DataFrame para mayor control o para optimizar.

In [ ]:
import polars as pl

datos_para_schema = {
    'col_a': [1, 2, 3],
    'col_b': [True, False, True],
    'col_c': ['x', 'y', 'z']
}
mi_schema = {
    'col_a': pl.Int16, # Usamos un entero más pequeño
    'col_b': pl.Boolean,
    'col_c': pl.Utf8
}
df_con_schema = pl.DataFrame(data=datos_para_schema, schema=mi_schema)
print("\nDataFrame de Polars con schema explícito:")
print(df_con_schema)
print("\nSchema resultante:")
print(df_con_schema.schema)

3. **Desde una lista de diccionarios (usando pl.from_dicts()):**

* Cada diccionario representa una fila.
* Polars manejará las claves faltantes introduciendo valores nulos (null en Polars, análogo a NaN en Pandas para números o None para objetos).

In [ ]:
import polars as pl

lista_de_diccionarios_pl = [
    {'Nombre': 'Carlos', 'Edad': 25, 'Profesion': 'Ingeniero'},
    {'Nombre': 'Laura', 'Edad': 30, 'Profesion': 'Doctora'},
    {'Nombre': 'Pedro', 'Edad': 22, 'Ciudad': 'Bogotá'} # Clave 'Ciudad' nueva, 'Profesion' puede faltar
]
df_desde_lista_dic_pl = pl.from_dicts(lista_de_diccionarios_pl)
print("\nDataFrame de Polars desde una lista de diccionarios:")
print(df_desde_lista_dic_pl)
print("\nSchema resultante:")
print(df_desde_lista_dic_pl.schema)

4. **Leyendo desde archivos (adelanto):**

Al igual que Pandas, Polars puede leer de varios formatos. Suelen ser más rápidas y eficientes, especialmente `read_parquet` y `scan_csv` (para modo lazy).

```
# Conceptualmente (no se ejecutará sin un archivo real):
df_csv_polars = pl.read_csv("ruta/a/tu/archivo.csv")
df_parquet_polars = pl.read_parquet("ruta/a/tu/archivo.parquet")

# Usando scan_csv para modo lazy (se explicará más adelante):
df_lazy_csv = pl.scan_csv("ruta/a/tu/archivo_grande.csv")
df_resultado = df_lazy_csv.filter(pl.col("columna") > 10).collect()
```

---

# **Inspección y Exploración Básica de Datos**

Al igual que con Pandas, una vez que tienes un DataFrame en Polars (ya sea creado o cargado), querrás entender su estructura, los tipos de datos que contiene, y obtener una visión general de su contenido. Polars ofrece un conjunto de atributos y métodos muy eficientes para esto.

Primero, creemos un DataFrame de Polars para nuestros ejemplos de inspección. Lo haremos similar al que usamos con Pandas para que puedas comparar, pero usando las convenciones de Polars.

In [ ]:
import polars as pl
from datetime import date, datetime, timedelta # Para crear fechas y datetimes

# Creando un DataFrame de Polars de ejemplo
datos_polars_inspeccion = {
    'ID_Sensor': [1, 2, 3, 4, 5, 6, 7, 8],
    'Tipo_Sensor': ['Temp', 'Hum', 'Pres', 'Temp', 'Luz', 'Hum', 'Temp', None], # String con un None
    'Lectura_Valor': [22.5, 65.2, 1012.5, 23.1, 800.0, 66.0, None, 10.5], # Float con un None
    'Fecha_Lectura': [
        date(2025, 5, 10), date(2025, 5, 10), date(2025, 5, 11), date(2025, 5, 11),
        date(2025, 5, 12), date(2025, 5, 12), date(2025, 5, 13), date(2025, 5, 13)
    ],
    'Hora_Lectura': [
        datetime(2025, 5, 10, 10, 0, 0), datetime(2025, 5, 10, 10, 5, 0),
        datetime(2025, 5, 11, 14, 30, 0), datetime(2025, 5, 11, 14, 32, 0),
        datetime(2025, 5, 12, 8, 0, 0), None, # Datetime con un None
        datetime(2025, 5, 13, 18, 0, 0), datetime(2025, 5, 13, 18, 3, 0)
    ],
    'Activo': [True, True, False, True, True, False, True, False]
}

# Definimos el schema para mejor control de tipos, especialmente para Date y Datetime
schema_definido = {
    'ID_Sensor': pl.Int32,
    'Tipo_Sensor': pl.Utf8,      # Tipo para strings
    'Lectura_Valor': pl.Float32,
    'Fecha_Lectura': pl.Date,
    'Hora_Lectura': pl.Datetime, # Acepta microsegundos por defecto, podemos especificar 'us' o 'ms'
    'Activo': pl.Boolean
}

df_pl_ins = pl.DataFrame(datos_polars_inspeccion, schema=schema_definido)

print("DataFrame de Polars para inspección:")
print(df_pl_ins)
print("\n" + "="*50 + "\n")

## 1. **`head(n)` y `tail(n)`: Ver las primeras/últimas filas:**
Funcionan de manera muy similar a Pandas. Por defecto muestran 5 filas.

In [ ]:
print("Primeras 3 filas del DataFrame (df_pl_ins.head(3)):")
print(df_pl_ins.head(3))

print("\nÚltimas 2 filas del DataFrame (df_pl_ins.tail(2)):")
print(df_pl_ins.tail(2))
print("\n" + "="*50 + "\n")

## 2. **`shape`: Dimensiones del DataFrame**
Un atributo que devuelve una tupla (número_de_filas, número_de_columnas).

In [ ]:
print("Dimensiones del DataFrame (filas, columnas) (df_pl_ins.shape):")
print(df_pl_ins.shape)
print(f"El DataFrame tiene {df_pl_ins.shape[0]} filas y {df_pl_ins.shape[1]} columnas.")

# También puedes obtener alto y ancho por separado:
print(f"Altura (número de filas): {df_pl_ins.height}")
print(f"Anchura (número de columnas): {df_pl_ins.width}")
print("\n" + "="*50 + "\n")

## 3. **`describe()`: Estadísticas descriptivas**

Proporciona un resumen estadístico de las columnas. Para columnas numéricas, incluye count (no nulos), null_count, mean, std, min, percentiles (25%, 50%, 75%) y max. Para columnas no numéricas (como strings), muestra count, null_count, unique, y mode (el valor más frecuente).



In [ ]:
print("Estadísticas descriptivas del DataFrame (df_pl_ins.describe()):")
print(df_pl_ins.describe())
# Nota: Polars describe() intenta ser inteligente y puede mostrar diferentes cosas
# según el tipo de dato de la columna, incluyendo strings.
print("\n" + "="*50 + "\n")


"""
La salida de describe() en Polars es bastante completa y se adapta bien a diferentes tipos de datos.
"""

## 4. **`dtypes`: Tipos de datos de cada columna**
Un atributo que devuelve una lista de los tipos de datos de Polars para cada columna.

In [ ]:
print("Tipos de datos de cada columna (df_pl_ins.dtypes):")
print(df_pl_ins.dtypes)
print("\n" + "="*50 + "\n")

"""
Salida Esperada:

[Int32, Utf8, Float32, Date, Datetime(time_unit='us', time_zone=None), Boolean]

"""

## 5. **`schema`: Schema del DataFrame**
Un atributo que devuelve un diccionario ordenado (o similar) mapeando los nombres de las columnas a sus tipos de datos de Polars. Es más detallado que dtypes porque muestra los nombres de las columnas junto con los tipos.

In [ ]:
print("Schema del DataFrame (df_pl_ins.schema):")
print(df_pl_ins.schema)
print("\n" + "="*50 + "\n")

"""
Salida Esperada:

OrderedDict({'ID_Sensor': Int32, 'Tipo_Sensor': Utf8, 'Lectura_Valor': Float32, 'Fecha_Lectura': Date, 'Hora_Lectura': Datetime(time_unit='us', time_zone=None), 'Activo': Boolean})
"""

## 6. **`columns`: Nombres de las columnas:**
Un atributo que devuelve una lista con los nombres de todas las columnas.

In [ ]:
print("Nombres de las columnas (df_pl_ins.columns):")
print(df_pl_ins.columns)
print("\n" + "="*50 + "\n")

## 7. **`glimpse()`: Un vistazo rápido al DataFrame:**
Este método es específico de Polars y es muy útil. Muestra una vista transpuesta del DataFrame, listando cada columna, su tipo de dato, y los primeros valores. Es como una combinación de info() y head() pero más compacto y orientado a la estructura.

In [ ]:
print("Vistazo al DataFrame (df_pl_ins.glimpse()):")
df_pl_ins.glimpse() # glimpse() imprime directamente, no devuelve un objeto para imprimir
print("\n" + "="*50 + "\n")

"""
  Salida Esperada:

Vistazo al DataFrame (df_pl_ins.glimpse()):
Rows: 8
Columns: 6
$ ID_Sensor     <i32> 1, 2, 3, 4, 5, 6, 7, 8
$ Tipo_Sensor   <str> 'Temp', 'Hum', 'Pres', 'Temp', 'Luz', 'Hum', 'Temp', None
$ Lectura_Valor <f32> 22.5, 65.2, 1012.5, 23.1, 800.0, 66.0, None, 10.5
$ Fecha_Lectura <date> 2025-05-10, 2025-05-10, 2025-05-11, 2025-05-11, 2025-05-12, 2025-05-12, 2025-05-13, 2025-05-13
$ Hora_Lectura  <datetime[μs]> 2025-05-10 10:00:00, 2025-05-10 10:05:00, 2025-05-11 14:30:00, 2025-05-11 14:32:00, 2025-05-12 08:00:00, None, 2025-05-13 18:00:00, 2025-05-13 18:03:00
$ Activo        <bool> true, true, false, true, true, false, true, false

"""

## 8. **`is_empty()`: Verificar si el DataFrame está vacío**

In [ ]:
print(f"¿El DataFrame está vacío? (df_pl_ins.is_empty()): {df_pl_ins.is_empty()}")
df_vacio = pl.DataFrame()
print(f"¿Un DataFrame vacío lo está? (df_vacio.is_empty()): {df_vacio.is_empty()}")
print("\n" + "="*50 + "\n")

## 9. **Operaciones en Series: `is_unique()`, `n_unique()`, `value_counts()`**
Estas operaciones se aplican a una columna (Serie) del DataFrame.

In [ ]:
# Seleccionar una columna (Serie)
col_tipo_sensor = df_pl_ins['Tipo_Sensor']
print(f"Columna 'Tipo_Sensor':\n{col_tipo_sensor}")

# ¿Son todos los valores en 'Tipo_Sensor' únicos?
print(f"\n¿Son únicos los valores en 'Tipo_Sensor'? {col_tipo_sensor.is_unique().all()}") # is_unique() devuelve una serie booleana, .all() la reduce

# Número de valores únicos en 'Tipo_Sensor'
print(f"Número de valores únicos en 'Tipo_Sensor': {col_tipo_sensor.n_unique()}")

# Frecuencia de cada valor en 'Tipo_Sensor'
print("\nFrecuencia de valores en 'Tipo_Sensor' (value_counts()):")
print(col_tipo_sensor.value_counts())
print("\n" + "="*50 + "\n")

"""
  Salida Esperada:

Frecuencia de valores en 'Tipo_Sensor' (value_counts()):
shape: (5, 2)
┌─────────────┬────────┐
│ Tipo_Sensor ┆ counts │
│ ---         ┆ ---    │
│ str         ┆ u32    │
╞═════════════╪════════╡
│ Temp        ┆ 3      │
│ Hum         ┆ 2      │
│ Luz         ┆ 1      │
│ null        ┆ 1      │
│ Pres        ┆ 1      │
└─────────────┴────────┘

"""

---
## **Selección y Filtrado de Datos**

Esta es una de las áreas donde Polars realmente muestra su poder y su enfoque diferente, principalmente a través de su API de Expresiones. Las expresiones te permiten construir operaciones complejas de forma declarativa, que Polars luego puede optimizar, especialmente en modo lazy (aunque también funcionan en modo eager)

Seguimos usando el mismo dataset:

In [ ]:
import polars as pl
from datetime import date, datetime # Para nuestros datos de ejemplo

# Recreamos el DataFrame anterior para consistencia
datos_polars_inspeccion = {
    'ID_Sensor': [1, 2, 3, 4, 5, 6, 7, 8],
    'Tipo_Sensor': ['Temp', 'Hum', 'Pres', 'Temp', 'Luz', 'Hum', 'Temp', None],
    'Lectura_Valor': [22.5, 65.2, 1012.5, 23.1, 800.0, 66.0, None, 10.5],
    'Fecha_Lectura': [
        date(2025, 5, 10), date(2025, 5, 10), date(2025, 5, 11), date(2025, 5, 11),
        date(2025, 5, 12), date(2025, 5, 12), date(2025, 5, 13), date(2025, 5, 13)
    ],
    'Hora_Lectura': [
        datetime(2025, 5, 10, 10, 0, 0), datetime(2025, 5, 10, 10, 5, 0),
        datetime(2025, 5, 11, 14, 30, 0), datetime(2025, 5, 11, 14, 32, 0),
        datetime(2025, 5, 12, 8, 0, 0), None,
        datetime(2025, 5, 13, 18, 0, 0), datetime(2025, 5, 13, 18, 3, 0)
    ],
    'Activo': [True, True, False, True, True, False, True, False]
}
schema_definido = {
    'ID_Sensor': pl.Int32, 'Tipo_Sensor': pl.Utf8, 'Lectura_Valor': pl.Float32,
    'Fecha_Lectura': pl.Date, 'Hora_Lectura': pl.Datetime, 'Activo': pl.Boolean
}
df_pl_ins = pl.DataFrame(datos_polars_inspeccion, schema=schema_definido)

print("DataFrame original Selección y Filtrado de Datos:")
print(df_pl_ins)
print("\n" + "="*50 + "\n")

## 1. **Selección de Columnas**

  ### **Usando `df.select()` con Expresiones `pl.col()` (la forma más idiomática y potente):**

  El método `select()` es la principal forma de elegir, transformar o crear nuevas columnas. Se usa con expresiones de Polars, comúnmente pl.col("nombre_columna") para referirse a una columna existente.

In [ ]:
print("--- Selección de Columnas con df.select() ---")
# Seleccionar una sola columna
df_una_columna = df_pl_ins.select(pl.col("Tipo_Sensor"))
print("\nSelección de una columna ('Tipo_Sensor'):")
print(df_una_columna)

# Seleccionar múltiples columnas
df_multiples_columnas = df_pl_ins.select(
    pl.col("ID_Sensor"),
    pl.col("Fecha_Lectura"),
    pl.col("Lectura_Valor")
)
print("\nSelección de múltiples columnas:")
print(df_multiples_columnas)

# Seleccionar columnas y renombrarlas con alias
df_con_alias = df_pl_ins.select(
    pl.col("ID_Sensor").alias("Identificador"),
    pl.col("Lectura_Valor").alias("Valor")
)
print("\nSelección con alias:")
print(df_con_alias)

# Seleccionar todas las columnas
df_todas_columnas = df_pl_ins.select(pl.all()) # pl.all() selecciona todo
# print("\nTodas las columnas (igual que el original en este caso):")
# print(df_todas_columnas)

# Excluir columnas
df_sin_fechas = df_pl_ins.select(pl.all().exclude("Fecha_Lectura", "Hora_Lectura"))
print("\nTodas las columnas EXCEPTO las de fecha:")
print(df_sin_fechas)
print("\n" + "="*50 + "\n")

### **Usando `df.select()` con strings o listas de strings (más conciso para selección simple):**
Polars permite pasar directamente strings o listas de strings a select para una selección simple.

In [ ]:
# Seleccionar una columna pasando su nombre como string
df_tipo_sensor_str = df_pl_ins.select("Tipo_Sensor")
print("Selección de 'Tipo_Sensor' usando string:")
print(df_tipo_sensor_str)

# Seleccionar múltiples columnas pasando una lista de strings
df_id_valor_str = df_pl_ins.select(["ID_Sensor", "Lectura_Valor", "Activo"])
print("\nSelección de múltiples columnas usando lista de strings:")
print(df_id_valor_str)
print("\n" + "="*50 + "\n")

### **Usando Selectores (API `pl.selectors` o cs):**
Para selecciones más programáticas o basadas en patrones/tipos.

In [ ]:
import polars.selectors as cs # Alias común para los selectores

# Seleccionar todas las columnas de tipo string (Utf8)
df_columnas_string = df_pl_ins.select(cs.string()) # cs.string() es un alias para cs.by_dtype(pl.Utf8)
print("Selección de todas las columnas de tipo string (Utf8):")
print(df_columnas_string)

# Seleccionar todas las columnas numéricas (enteros y flotantes)
df_columnas_numericas = df_pl_ins.select(cs.numeric())
print("\nSelección de todas las columnas numéricas:")
print(df_columnas_numericas)

# Seleccionar columnas que empiezan con "Fecha_"
df_columnas_fecha = df_pl_ins.select(cs.starts_with("Fecha_"))
print("\nSelección de columnas que empiezan con 'Fecha_':")
print(df_columnas_fecha)
print("\n" + "="*50 + "\n")

### **Acceso directo con corchetes [ ] (similar a Pandas):**

* `df["nombre_columna"]` devuelve una Serie de Polars.
* `df[["col1", "col2"]]` devuelve un DataFrame de Polars con esas columnas.

In [ ]:
# Seleccionar una columna como Serie
serie_tipo_sensor = df_pl_ins["Tipo_Sensor"]
print("Columna 'Tipo_Sensor' como Serie de Polars:")
print(serie_tipo_sensor)
print(f"Tipo: {type(serie_tipo_sensor)}")

# Seleccionar múltiples columnas como DataFrame
df_id_y_activo = df_pl_ins[["ID_Sensor", "Activo"]]
print("\nColumnas 'ID_Sensor' y 'Activo' como DataFrame:")
print(df_id_y_activo)
print(f"Tipo: {type(df_id_y_activo)}")
print("\n" + "="*50 + "\n")

## 2. **Selección de Filas (Slicing)**

Polars no tiene un índice como el de Pandas que se use con .loc o .iloc de la misma manera para la selección de filas basada en etiquetas. La selección de filas se hace principalmente por posición o condición.

### **Usando d`f.slice(offset, length)`:**
Selecciona un trozo del DataFrame especificando un desplazamiento (desde dónde empezar) y una longitud (cuántas filas tomar).



> **Nota:** Recuerda que `df.head(n)` y `df.tail(n)` también son formas de seleccionar las primeras o últimas n filas.



In [ ]:
print("--- Selección de Filas (Slicing) ---")
# Seleccionar 3 filas empezando desde la fila en posición 2 (la tercera fila)
df_slice = df_pl_ins.slice(offset=2, length=3)
print("\nSlice de 3 filas desde la posición 2:")
print(df_slice)

# Seleccionar las primeras 4 filas (similar a head(4) pero con slice)
df_slice_inicio = df_pl_ins.slice(offset=0, length=4)
# print("\nSlice de las primeras 4 filas:")
# print(df_slice_inicio)
print("\n" + "="*50 + "\n")

### **Acceso directo a filas por índice entero (similar a iloc para una sola fila en Pandas):**

* `df[indice_fila] `devuelve la fila como un DataFrame de una fila.
* `df[indice_inicio:indice_fin]` realiza un slicing.

In [ ]:
# Seleccionar la fila en la posición 0 (la primera fila)
fila_0 = df_pl_ins[0]
print("Fila en posición 0:")
print(fila_0)

# Slicing de filas (similar a Python, el final es exclusivo)
slice_filas_directo = df_pl_ins[2:5] # Filas en posición 2, 3, 4
print("\nSlice de filas [2:5]:")
print(slice_filas_directo)
print("\n" + "="*50 + "\n")

## 3. **Filtrado de Filas con `df.filter()` y Expresiones**

Este es el método principal y más potente para seleccionar filas que cumplen ciertas condiciones. Se usa con la API de Expresiones de Polars.

In [ ]:
print("--- Filtrado de Filas con df.filter() ---")
# 1. Condición numérica: Sensores con Lectura_Valor > 50.0
df_lecturas_altas = df_pl_ins.filter(pl.col("Lectura_Valor") > 50.0)
print("\nSensores con Lectura_Valor > 50.0:")
print(df_lecturas_altas)

# 2. Condición de string: Sensores de tipo 'Temp'
df_tipo_temp = df_pl_ins.filter(pl.col("Tipo_Sensor") == "Temp")
print("\nSensores de tipo 'Temp':")
print(df_tipo_temp)

# También puedes usar métodos de string dentro de las expresiones
df_tipo_contiene_m = df_pl_ins.filter(pl.col("Tipo_Sensor").str.contains("m")) # Contiene 'm' (Hum)
print("\nSensores cuyo tipo contiene la letra 'm':")
print(df_tipo_contiene_m)

# 3. Combinar múltiples condiciones (usando & para AND, | para OR, ~ para NOT):
# Sensores de tipo 'Temp' Y que estén 'Activo'
df_temp_activos = df_pl_ins.filter(
    (pl.col("Tipo_Sensor") == "Temp") & (pl.col("Activo") == True) # o simplemente pl.col("Activo")
)
print("\nSensores de tipo 'Temp' Y Activos:")
print(df_temp_activos)

# Sensores con Lectura_Valor < 20 O Lectura_Valor > 500
df_lecturas_extremos = df_pl_ins.filter(
    (pl.col("Lectura_Valor") < 20.0) | (pl.col("Lectura_Valor") > 500.0)
)
print("\nSensores con Lectura_Valor < 20.0 O > 500.0:")
print(df_lecturas_extremos)

# 4. Usando `.is_in()`:
# Sensores de tipo 'Hum' o 'Luz'
tipos_buscados = ["Hum", "Luz"]
df_hum_o_luz = df_pl_ins.filter(pl.col("Tipo_Sensor").is_in(tipos_buscados))
print("\nSensores de tipo 'Hum' o 'Luz':")
print(df_hum_o_luz)

# 5. Usando `.is_null()` o `.is_not_null()`:
# Filas donde 'Lectura_Valor' es nulo
df_lectura_nula = df_pl_ins.filter(pl.col("Lectura_Valor").is_null())
print("\nFilas con 'Lectura_Valor' nulo:")
print(df_lectura_nula)

# Filas donde 'Tipo_Sensor' NO es nulo
df_tipo_no_nulo = df_pl_ins.filter(pl.col("Tipo_Sensor").is_not_null())
print("\nFilas con 'Tipo_Sensor' NO nulo:")
print(df_tipo_no_nulo)
print("\n" + "="*50 + "\n")

## 4. **Encadenamiento de Operaciones (Selección y Filtrado)**

Una gran ventaja de Polars es cómo se pueden encadenar las operaciones de forma legible:

In [ ]:
print("--- Encadenamiento de Operaciones ---")
# Seleccionar ID_Sensor, Tipo_Sensor y Lectura_Valor para los sensores 'Temp' que estén activos
# y cuya lectura sea mayor a 23.0
resultado_encadenado = (
    df_pl_ins
    .filter(
        (pl.col("Tipo_Sensor") == "Temp") &
        (pl.col("Activo") == True) &
        (pl.col("Lectura_Valor") > 23.0)
    )
    .select(["ID_Sensor", "Tipo_Sensor", "Lectura_Valor"])
    .sort("Lectura_Valor", descending=True) # Añadimos un ordenamiento también
)
print("\nResultado de filtrar y seleccionar encadenadamente:")
print(resultado_encadenado)

---
## **Manipulación de Datos.**

Aquí es donde la "Expression API" de Polars realmente se luce. La mayoría de las transformaciones, creación de nuevas columnas y modificaciones se realizan de manera muy eficiente y expresiva usando el método `with_columns()` (o a` veces select()` si solo quieres un subconjunto transformado).

* DataFrame para ejemplos de manipulación:

In [ ]:
import polars as pl
from datetime import date, datetime, timedelta
import numpy as np # Para algún NaN si es necesario

# DataFrame de Polars para manipulación
datos_manipulacion = {
    'ID_Transaccion': range(1, 9),
    'Producto': ['A', 'B', 'A', 'C', 'B', 'A', 'C', 'B'],
    'Fecha': [
        date(2025, 1, 5), date(2025, 1, 5), date(2025, 1, 6), date(2025, 1, 6),
        date(2025, 1, 7), date(2025, 1, 8), date(2025, 1, 8), date(2025, 1, 9)
    ],
    'Cantidad': [10, 5, 8, 12, 6, 15, 7, 9],
    'Precio_Unitario': [100.0, 250.5, 105.0, 50.0, 260.0, None, 52.5, 240.0],   # Incluimos un None
    'Descuento_Aplicado': [0.1, 0.0, 0.1, 0.05, 0.0, 0.15, 0.0, 0.05]
}
schema_manip = {
    'ID_Transaccion': pl.Int16, 'Producto': pl.Categorical,                     # Usamos Categorical para Producto
    'Fecha': pl.Date, 'Cantidad': pl.Int32, 'Precio_Unitario': pl.Float64,
    'Descuento_Aplicado': pl.Float32
}
df_pl_manip = pl.DataFrame(datos_manipulacion, schema=schema_manip)

"""
  Usamos pl.Categorical para la columna 'Producto', que es un tipo de dato eficiente en Polars para strings con un número limitado de valores únicos.
"""

print("DataFrame original para Manipulación de Datos:")
print(df_pl_manip)
print("\n" + "="*50 + "\n")

### 1. **Añadir y Modificar Columnas con `df.with_columns()`**

`with_columns()` es el método principal para añadir nuevas columnas o transformar las existentes. Toma una o más expresiones de Polars.

 * Añadir una columna con un valor literal (escalar):
  Usamos `pl.lit()` para crear una expresión a partir de un valor literal.

In [ ]:
df_con_literal = df_pl_manip.with_columns(
    pl.lit("Tienda_Online").alias("Canal_Venta")
)
print("--- Añadir/Modificar Columnas con with_columns ---")
print("\nDataFrame con nueva columna 'Canal_Venta' (literal):")
print(df_con_literal)

## 2. **Añadir una columna derivada de otras existentes:**

In [ ]:
# Calcular Ingreso_Bruto = Cantidad * Precio_Unitario
# Calcular Precio_Neto = Precio_Unitario * (1 - Descuento_Aplicado)
df_con_calculos = df_pl_manip.with_columns([
    (pl.col("Cantidad") * pl.col("Precio_Unitario")).alias("Ingreso_Bruto"),
    (pl.col("Precio_Unitario") * (1 - pl.col("Descuento_Aplicado"))).alias("Precio_Neto_Unitario") # Observa que si 'Precio_Unitario' es nulo, el resultado de las operaciones también será nulo.
])
print("\nDataFrame con 'Ingreso_Bruto' y 'Precio_Neto_Unitario':")
print(df_con_calculos)

## 3. **Modificar una columna existente:**
Si el alias (`.alias("nombre_columna")`) coincide con el nombre de una columna existente, esa columna se sobrescribe.

In [ ]:
# Convertir 'Producto' a minúsculas (ya es Categorical, pero .str funciona en expresiones)
df_producto_minusculas = df_pl_manip.with_columns(
    pl.col("Producto").str.to_lowercase().alias("Producto") # Sobrescribe 'Producto'
)
print("\nDataFrame con 'Producto' en minúsculas:")
print(df_producto_minusculas)

## 4. **Crear columnas con lógica condicional (`pl.when().then().otherwise()`):**

In [ ]:
# Categorizar Descuento: 'Alto' si > 0.1, 'Medio' si > 0.05, 'Bajo' de lo contrario
df_con_categoria_descuento = df_pl_manip.with_columns(
    pl.when(pl.col("Descuento_Aplicado") > 0.1)
    .then(pl.lit("Alto"))
    .when(pl.col("Descuento_Aplicado") > 0.05)
    .then(pl.lit("Medio"))
    .otherwise(pl.lit("Bajo"))
    .alias("Nivel_Descuento")
)
print("\nDataFrame con 'Nivel_Descuento':")
print(df_con_categoria_descuento[["Producto", "Descuento_Aplicado", "Nivel_Descuento"]])
print("\n" + "="*50 + "\n")

## **Eliminar Columnas con `df.drop()`**

In [ ]:
print("--- Eliminar Columnas ---")
# Creamos una copia para no afectar el df original de esta sección
df_para_drop = df_con_calculos.clone() # .clone() para una copia profunda

# Eliminar una sola columna
df_drop_una = df_para_drop.drop("Ingreso_Bruto")
print("\nDataFrame después de eliminar 'Ingreso_Bruto':")
print(df_drop_una.columns)

# Eliminar múltiples columnas
df_drop_multiples = df_para_drop.drop(["Ingreso_Bruto", "Precio_Neto_Unitario"])
print("\nDataFrame después de eliminar 'Ingreso_Bruto' y 'Precio_Neto_Unitario':")
print(df_drop_multiples.columns)

# También se puede pasar una sola columna sin lista a drop
# df_drop_una_alt = df_para_drop.drop("Ingreso_Bruto")
print("\n" + "="*50 + "\n")

## **C. Manejo de Valores Nulos (null)**

## 1. **Contar nulos (repaso):**

In [ ]:
print("--- Manejo de Valores Nulos ---")
print("Conteo de nulos por columna:")
print(df_pl_manip.null_count()) # Muestra un DataFrame con los conteos
# O para una columna específica, dentro de una expresión:
# print(df_pl_manip.select(pl.col("Precio_Unitario").is_null().sum().alias("nulos_precio")))

## 2. **Eliminar filas con nulos: `df.drop_nulls()`:**

In [ ]:
# Eliminar cualquier fila que tenga al menos un valor nulo
df_sin_nulos_filas = df_pl_manip.drop_nulls()
print("\nDataFrame después de drop_nulls() (elimina filas con cualquier nulo):")
print(df_sin_nulos_filas) # La fila con Precio_Unitario nulo desaparece

# Eliminar filas si tienen nulos en un subconjunto de columnas
# (En nuestro df, solo Precio_Unitario tiene nulos)
df_sin_nulos_subset = df_pl_manip.drop_nulls(subset=["Precio_Unitario"])
print("\nDataFrame después de drop_nulls(subset=['Precio_Unitario']):")
print(df_sin_nulos_subset)

## 3. **Rellenar nulos usando expresiones con fill_null() (dentro de with_columns o select):**

* Tambien existen otras estrategías para fill_null: '`backward`', '`min`', '`max`', '`one`', '`zero`'.

In [ ]:
# Rellenar nulos en 'Precio_Unitario' con un valor literal (ej. 0.0)
df_relleno_literal = df_pl_manip.with_columns(
    pl.col("Precio_Unitario").fill_null(0.0).alias("Precio_Rellenado_Literal")
)
print("\n'Precio_Unitario' con nulos rellenados con 0.0:")
print(df_relleno_literal[["Precio_Unitario", "Precio_Rellenado_Literal"]])

# Rellenar nulos con la media de la columna (Polars calcula la media ignorando nulos)
df_relleno_media = df_pl_manip.with_columns(
    pl.col("Precio_Unitario").fill_null(pl.col("Precio_Unitario").mean()).alias("Precio_Rellenado_Media")
)
print("\n'Precio_Unitario' con nulos rellenados con la media:")
print(df_relleno_media[["ID_Transaccion", "Precio_Unitario", "Precio_Rellenado_Media"]])

# Rellenar nulos con una estrategia (ej. 'forward' fill - ffill)
# Ordenamos por fecha primero para que el forward fill tenga más sentido si hubiera múltiples nulos
df_relleno_forward = df_pl_manip.sort("Fecha").with_columns(
    pl.col("Precio_Unitario").fill_null(strategy="forward").alias("Precio_Rellenado_Forward")
)
print("\n'Precio_Unitario' con nulos rellenados con estrategia 'forward' (después de ordenar):")
print(df_relleno_forward[["ID_Transaccion", "Fecha", "Precio_Unitario", "Precio_Rellenado_Forward"]])
print("\n" + "="*50 + "\n")

## **Renombrar Columnas con `df.rename()`:**

In [ ]:
print("--- Renombrar Columnas ---")
df_renombrado = df_pl_manip.rename({
    "ID_Transaccion": "ID",
    "Precio_Unitario": "Precio_Base"
})
print("\nDataFrame con columnas renombradas:")
print(df_renombrado.columns)
print(df_renombrado.head(2))
print("\n" + "="*50 + "\n")

## **Cambiar Tipos de Datos (Casting) con `cast()`**:
Se usa como una expresión dentro de `with_columns` o `select`.

In [ ]:
print("--- Cambiar Tipos de Datos (Casting) ---")
# Cambiar Cantidad a Float32 y Precio_Unitario (después de rellenar nulos) a Int32
df_tipos_cambiados = df_pl_manip.with_columns([
    pl.col("Cantidad").cast(pl.Float32),
    pl.col("Precio_Unitario").fill_null(0).cast(pl.Int32).alias("Precio_Entero_NoNulo") # Encadenamos fill_null y cast
])
print("\nDataFrame con tipos de datos cambiados:")
print(df_tipos_cambiados.select(["Producto", "Cantidad", "Precio_Entero_NoNulo"]))
print(df_tipos_cambiados.dtypes) # Mostrar los nuevos tipos
print("\n" + "="*50 + "\n")

## **Ordenar Datos con `df.sort()`**

In [ ]:
print("--- Ordenar Datos ---")
# Ordenar por 'Fecha' (ascendente) y luego por 'Precio_Unitario' (descendente)
df_ordenado = df_pl_manip.sort(["Fecha", "Precio_Unitario"], descending=[False, True])
print("\nDataFrame ordenado por Fecha (asc) y Precio_Unitario (desc):")
print(df_ordenado)
print("\n" + "="*50 + "\n")

## **Aplicar Funciones Personalizadas con map_elements() (en Series/Expresiones):**

Si necesitas aplicar una función Python arbitraria que no tiene un equivalente directo en las expresiones de Polars, puedes usar `map_elements()`. Ten en cuenta que esto puede ser más lento que usar expresiones nativas de Polars porque implica pasar datos entre el motor de Rust y el intérprete de Python para cada elemento.

> **Nota:** `map_elements` reemplazó a `.apply()` para Series en versiones más recientes de Polars para este tipo de operación. Asegúrate de usar return_dtype.

In [ ]:
print("--- Aplicar Funciones Personalizadas con map_elements ---")
def mi_funcion_compleja_producto(nombre_producto: str) -> str:
    if nombre_producto is None:
        return "PRODUCTO DESCONOCIDO"
    return f"PROD-{nombre_producto.upper()}-XYZ"

# Es crucial especificar el return_dtype para map_elements
df_con_map = df_pl_manip.with_columns(
    pl.col("Producto")
    .map_elements(mi_funcion_compleja_producto, return_dtype=pl.Utf8)
    .alias("Producto_Transformado")
)
print("\nDataFrame con 'Producto_Transformado' usando map_elements:")
print(df_con_map.select(["Producto", "Producto_Transformado"]))
print("\n" + "="*50 + "\n")

---

# **5. Agrupación de Datos (`group_by`) y Agregaciones.**

Al igual que en Pandas, la agrupación te permite dividir tus datos en subconjuntos basados en los valores de ciertas columnas y luego realizar cálculos o aplicar transformaciones a cada uno de estos subconjuntos. El paradigma sigue siendo Split-Apply-Combine.

En Polars, esto se logra principalmente con el método group_by() seguido del **`método agg()`**, donde la API de Expresiones juega un papel central para definir las agregaciones.

In [ ]:
import polars as pl
from datetime import date

# DataFrame de Polars para agrupación y agregaciones
datos_agrupacion = {
    'ID_Transaccion': range(1, 11),
    'Producto': ['A', 'B', 'A', 'C', 'B', 'A', 'C', 'B', 'A', 'B'],
    'Region': ['Norte', 'Sur', 'Norte', 'Este', 'Sur', 'Norte', 'Este', 'Sur', 'Oeste', 'Norte'],
    'Fecha': [
        date(2025, 1, 5), date(2025, 1, 5), date(2025, 1, 6), date(2025, 1, 6),
        date(2025, 1, 7), date(2025, 1, 8), date(2025, 1, 8), date(2025, 1, 9),
        date(2025, 1, 9), date(2025, 1, 10)
    ],
    'Cantidad': [10, 5, 8, 12, 6, 15, 7, 9, 20, 11],
    'Ingresos': [1000.0, 1252.5, 840.0, 600.0, 1560.0, 1575.0, 367.5, 2160.0, 2000.0, 2761.0],
    'Valoracion': [4, 5, 3, 4, 5, 2, 3, 5, 4, 5]
}
schema_agrup = {
    'ID_Transaccion': pl.Int16, 'Producto': pl.Categorical, 'Region': pl.Categorical,
    'Fecha': pl.Date, 'Cantidad': pl.Int32, 'Ingresos': pl.Float64,
    'Valoracion': pl.Int8
}
df_pl_agrup = pl.DataFrame(datos_agrupacion, schema=schema_agrup)

print("DataFrame original para aprupación de datos:")
print(df_pl_agrup)
print("\n" + "="*50 + "\n")

## 1. **Usando `df.group_by()`:**

El método `group_by()` por sí solo crea un objeto GroupBy. Para obtener un resultado tangible, necesitas encadenarle un método de agregación, típicamente `agg()`.

Agrupar por una sola columna:

In [ ]:
print("--- Agrupación con df.group_by() ---")
# Agrupar por 'Producto'. Esto crea un objeto GroupBy.
gb_producto = df_pl_agrup.group_by("Producto")
print("Tipo de objeto después de group_by:", type(gb_producto))
# <class 'polars.dataframe.group_by.GroupBy'>

## 2. **Agrupar por múltiples columnas:**
Pasando una lista de nombres de columnas.

In [ ]:
gb_producto_region = df_pl_agrup.group_by(["Producto", "Region"])
# print("\nTipo de objeto después de group_by múltiple:", type(gb_producto_region))

## 3. **Mantener el orden original de los grupos (`maintain_order=True`):**
Por defecto, group_by no garantiza el orden de los grupos resultantes. Si el orden es importante (por ejemplo, el orden en que aparecen los primeros grupos en tus datos originales), puedes usar `maintain_order=True.`

Esto puede tener un ligero impacto en el rendimiento.

In [ ]:
gb_producto_ordenado = df_pl_agrup.group_by("Producto", maintain_order=True)
gb_producto_ordenado

## **Aplicando Agregaciones con agg() y Expresiones**

El método `agg()` se aplica al objeto `GroupBy` y toma una lista de expresiones de Polars que definen qué y cómo agregar.

## 1. **Agregación simple en una columna:**

In [ ]:
# Calcular la suma total de 'Cantidad' para cada 'Producto'
suma_cantidad_por_producto = gb_producto.agg(
    pl.col("Cantidad").sum().alias("Cantidad_Total") # pl.col() para referenciar, .sum() para agregar, .alias() para nombrar
)
print("\nSuma de Cantidad por Producto:")
print(suma_cantidad_por_producto)

# Forma abreviada para agregaciones simples como sum, mean, min, max, etc.
# Polars también permite pl.sum("NombreColumna"), pl.mean("NombreColumna"), etc.
suma_ingresos_por_producto_corto = gb_producto.agg(
    pl.sum("Ingresos").alias("Ingresos_Totales")
)
print("\nSuma de Ingresos por Producto (forma corta):")
print(suma_ingresos_por_producto_corto)
print("\n" + "="*50 + "\n")

## 2. **Múltiples agregaciones sobre la misma columna:**

In [ ]:
metricas_ingresos_por_producto = gb_producto.agg([
    pl.col("Ingresos").sum().alias("Ingresos_Suma"),
    pl.col("Ingresos").mean().alias("Ingresos_Media"),
    pl.col("Ingresos").min().alias("Ingresos_Min"),
    pl.col("Ingresos").max().alias("Ingresos_Max"),
    pl.col("Ingresos").count().alias("Conteo_Transacciones_Ingresos") # Cuenta no nulos en Ingresos
])
print("Múltiples métricas de Ingresos por Producto:")
print(metricas_ingresos_por_producto)
print("\n" + "="*50 + "\n")

## 3. **Agregaciones sobre diferentes columnas:**
Simplemente listas todas las expresiones de agregación que necesitas

 > * **`pl.len()` vs `pl.count()`:**
    * `pl.len()` (o `pl.col("cualquier_columna").len()`) dentro de `agg` te da el número de filas en el grupo (similar a `size()` en Pandas `groups`).
    * `pl.count("nombre_columna")` (o `pl.col("nombre_columna").count()`) dentro de `agg` te da el número de valores no nulos en esa columna específica para el grupo. Si solo usas `pl.count()` sin especificar columna, actúa como `pl.len().` Para evitar ambigüedad, `pl.len()` es más claro para el tamaño del grupo.

In [ ]:
resumen_por_region = df_pl_agrup.group_by("Region", maintain_order=True).agg([
    pl.col("Cantidad").sum().alias("Cantidad_Total_Region"),
    pl.col("Ingresos").mean().alias("Ingresos_Promedio_Region"),
    pl.col("Valoracion").median().alias("Valoracion_Mediana_Region"),
    pl.col("Producto").n_unique().alias("Productos_Unicos_Region"), # Cuántos productos únicos hay por región
    pl.len().alias("Numero_Transacciones_Region") # pl.len() cuenta el número de filas en cada grupo
])
print("Resumen Agregado por Región:")
print(resumen_por_region)
print("\n" + "="*50 + "\n")

## 4. **Agregación Condicional:**
Puedes usar `.filter()` dentro de una expresión de agregación.

In [ ]:
# Suma de ingresos solo de valoraciones altas (>= 4) por Producto
ingresos_valoracion_alta_por_producto = df_pl_agrup.group_by("Producto").agg(
    pl.col("Ingresos").filter(pl.col("Valoracion") >= 4).sum().alias("Ingresos_Valoracion_Alta")
)
print("Suma de Ingresos de Valoraciones Altas (>=4) por Producto:")
print(ingresos_valoracion_alta_por_producto)
print("\n" + "="*50 + "\n")

## 5. **Otras expresiones de agregación útiles:**

* `.first()`: Primer elemento del grupo.
* `.last():` Último elemento del grupo.
* `.std()`: Desviación estándar.
* `.var()`: Varianza.
* `.quantile(q)`: Calcula el cuantil q.
* `.median()`: Equivalente a .quantile(0.5).

## **Pivot Tables con `df.pivot()` (Relacionado con Agrupación):**

Mientras que `group_by().agg() `te da una tabla donde los grupos son filas, `pivot()` te permite remodelar los datos de forma que algunos valores de una columna se conviertan en nuevas columnas, y se realiza una agregación. Es similar a las tablas dinámicas.

In [ ]:
print("--- Pivot Table ---")
# Queremos ver la suma de 'Cantidad' por cada 'Producto' (índice/filas)
# y para cada 'Region' (nuevas columnas)
pivot_cantidad_producto_region = df_pl_agrup.pivot(
    values="Cantidad",    # Columna cuyos valores llenarán la tabla
    index="Producto",     # Columna que se convertirá en el índice (filas)
    columns="Region",     # Columna cuyos valores únicos se convertirán en nuevas columnas
    aggregate_function="sum", # Cómo agregar 'Cantidad' si hay múltiples valores para una celda Producto-Region
    sort_columns=True     # Ordenar las nuevas columnas (Regiones)
)
print("Pivot Table: Suma de Cantidad por Producto y Región:")
print(pivot_cantidad_producto_region)

# Si no especificas aggregate_function y hay múltiples valores, tomará el primero.
# Es mejor ser explícito. Si quieres la media de ingresos:
pivot_media_ingresos = df_pl_agrup.pivot(
    values="Ingresos",
    index="Producto",
    columns="Region",
    aggregate_function="mean",
    sort_columns=True
)
print("\nPivot Table: Media de Ingresos por Producto y Región:")
print(pivot_media_ingresos) # Observa los nulls donde no hay combinación
print("\n" + "="*50 + "\n")

---
# Combinar y Fusionar diferentes DataFrames.

Al igual que con Pandas, en Polars a menudo necesitarás combinar datos que residen en diferentes DataFrames. Polars ofrece maneras eficientes y expresivas para lograr esto. Las operaciones principales son la concatenación (para "pegar" DataFrames) y los joins (para combinar DataFrames basándose en columnas clave, similar a SQL).

## 1. Concatenación con `pl.concat()`

La función pl.concat() se utiliza para unir múltiples DataFrames.

* Creando DataFrames de ejemplo para concatenar:

In [ ]:
import polars as pl

df_pl_c1 = pl.DataFrame({
    'id': [1, 2, 3],
    'nombre': ['Ana', 'Luis', 'Eva'],
    'valor_a': [10, 12, 14]
})

df_pl_c2 = pl.DataFrame({
    'id': [4, 5],
    'nombre': ['Juan', 'Sara'],
    'valor_a': [16, 18]
})

# DataFrame con una columna adicional y nombres ligeramente diferentes para mostrar manejo
df_pl_c3 = pl.DataFrame({
    'id': [6, 7],
    'nombre_completo': ['Carlos Rey', 'Laura Paz'], # Nombre de columna diferente
    'valor_a': [20, 22],
    'valor_b': [100, 102] # Nueva columna
})

print("df_pl_c1:")
print(df_pl_c1)
print("\ndf_pl_c2:")
print(df_pl_c2)
print("\ndf_pl_c3:")
print(df_pl_c3)
print("\n" + "="*50 + "\n")

## 2. **Concatenación Vertical (apilar filas, `how='vertical'` es una opción, por defecto intenta alinear):**

Polars intentará alinear las columnas por nombre. Si una columna existe en un DataFrame pero no en otro, se rellenará con null en el DataFrame donde falta.

> **Nota:** El comportamiento exacto y los mejores parámetros para how en pl.concat pueden evolucionar. `how='vertical_relaxed'` o `how='diagonal_relaxed'` son generalmente buenos para unir DataFrames con schemas que no son idénticos pero que deben apilarse.

In [ ]:
print("--- Concatenación Vertical ---")
# Concatenar df_pl_c1 y df_pl_c2 (mismas columnas)
df_concat_vertical_simple = pl.concat([df_pl_c1, df_pl_c2]) # how='vertical' es implícito
print("\nConcatenación vertical de df_pl_c1 y df_pl_c2:")
print(df_concat_vertical_simple)

# Concatenar los tres DataFrames. df_pl_c3 tiene columnas diferentes.
# Por defecto, `pl.concat` (cuando se infiere vertical) alinea por nombre y rellena con nulls.
df_concat_todos = pl.concat([df_pl_c1, df_pl_c2, df_pl_c3], how="diagonal_relaxed")
# `how="diagonal_relaxed"` o `how="vertical_relaxed"` son más explícitos
# para cuando los schemas no son idénticos y quieres que se unan inteligentemente.
# Simplemente `pl.concat` a menudo hace lo correcto si los tipos son compatibles o pueden ser promocionados.
# Para unir y rellenar columnas faltantes con null, `how='diagonal'` o `how='vertical_relaxed'` es robusto.
# A partir de versiones recientes, `how="vertical_relaxed"` es una buena opción para unir schemas que pueden no coincidir perfectamente.

print("\nConcatenación vertical de df_pl_c1, df_pl_c2, y df_pl_c3 (con `how='diagonal_relaxed'`):")
print(df_concat_todos)
# Verás 'nombre' y 'nombre_completo' como columnas separadas, y 'valor_b' con nulls para df_pl_c1 y df_pl_c2
print("\n" + "="*50 + "\n")

## 3. **Concatenación Horizontal `(how='horizontal')`:**
"Pega" los DataFrames uno al lado del otro, añadiendo columnas. Esto requiere que los DataFrames tengan el mismo número de filas.

In [ ]:
print("--- Concatenación Horizontal ---")
df_pl_h1 = pl.DataFrame({
    'A': ['A0', 'A1', 'A2'],
    'B': ['B0', 'B1', 'B2']
})
df_pl_h2 = pl.DataFrame({
    'C': ['C0', 'C1', 'C2'],
    'D': ['D0', 'D1', 'D2']
})
df_pl_h3_filas_distintas = pl.DataFrame({'E': ['E0', 'E1']})


df_concat_horizontal = pl.concat([df_pl_h1, df_pl_h2], how='horizontal')
print("\nConcatenación horizontal de df_pl_h1 y df_pl_h2:")
print(df_concat_horizontal)

# Esto daría un error porque el número de filas no coincide:
try:
    df_concat_horizontal_error = pl.concat([df_pl_h1, df_pl_h3_filas_distintas], how='horizontal')
except Exception as e:
    print(f"\nError al concatenar horizontalmente con diferente número de filas: {e}")
print("\n" + "="*50 + "\n")

## **Fusión tipo SQL con `df_left.join(df_right, ...)`:**

El método `.join()` en un DataFrame de Polars es la principal forma de realizar fusiones basadas en claves, similar a los JOIN de SQL.

## 1. **Creando DataFrames de ejemplo para join:**

In [ ]:
df_empleados = pl.DataFrame({
    'ID_Empleado': [1, 2, 3, 4, 5],
    'Nombre': ['Carlos', 'Laura', 'Pedro', 'Sofia', 'David'],
    'ID_Depto': [101, 102, 101, 103, 102]
})

df_departamentos = pl.DataFrame({
    'ID_Depto': [101, 102, 103, 104],
    'Nombre_Depto': ['Ventas', 'Marketing', 'IT', 'Recursos Humanos'],
    'Ubicacion': ['Piso 1', 'Piso 2', 'Piso 3', 'Piso 1']
})

df_proyectos = pl.DataFrame({
    'ID_Proyecto': ['P1', 'P2', 'P3', 'P4'],
    'ID_Empleado_Resp': [1, 2, 1, 5], # Usaremos esta para unir con ID_Empleado
    'Nombre_Proyecto': ['Alpha', 'Beta', 'Gamma', 'Delta']
})

print("df_empleados:")
print(df_empleados)
print("\ndf_departamentos:")
print(df_departamentos)
print("\ndf_proyectos:")
print(df_proyectos)
print("\n" + "="*50 + "\n")

## 2. **Tipos de Join (`how`):**
El método join toma el DataFrame "derecho" como primer argumento y luego los parámetros de unión.

* `how='inner'` (por defecto): Solo filas donde la clave de unión existe en ambos DataFrames.

In [ ]:
print("--- Joins tipo SQL ---")
# Inner join entre empleados y departamentos
df_inner_join = df_empleados.join(df_departamentos, on="ID_Depto")
print("\nInner Join de empleados y departamentos (en 'ID_Depto'):")
print(df_inner_join)
# Depto 104 (Recursos Humanos) no aparece porque ningún empleado está asignado a él.

  * `how='left'`: Todas las filas del DataFrame izquierdo (df_empleados en este caso) y las coincidentes del derecho.

In [ ]:
df_left_join = df_empleados.join(df_departamentos, on="ID_Depto", how="left")
print("\nLeft Join de empleados y departamentos:")
print(df_left_join)
# Todos los empleados aparecen. Si un ID_Depto no existe en df_departamentos, las cols de depto serán null.

  * `how='outer'`: Todas las filas de ambos DataFrames. Si no hay coincidencia, se rellena con null.

In [ ]:
df_outer_join = df_empleados.join(df_departamentos, on="ID_Depto", how="outer")
print("\nOuter Join de empleados y departamentos:")
print(df_outer_join)
# Todos los empleados y todos los departamentos aparecen.

  * `how='semi'`: Devuelve solo las filas del DataFrame izquierdo que tienen una coincidencia en el DataFrame derecho. No incluye columnas del DataFrame derecho.

In [ ]:
# Empleados que están en departamentos que SÍ existen en df_departamentos
# (En este caso, todos los empleados tienen un ID_Depto que existe en df_departamentos)
df_semi_join = df_empleados.join(df_departamentos, on="ID_Depto", how="semi")
print("\nSemi Join (empleados con departamento existente):")
print(df_semi_join)

  * `how='anti'`: Devuelve solo las filas del DataFrame *izquierdo* que NO tienen una coincidencia en el DataFrame *derecho*.

## 3. **Join con claves de nombres diferentes (`left_on`, `right_on`):**

In [ ]:
# Unir empleados con sus proyectos
# df_empleados tiene 'ID_Empleado', df_proyectos tiene 'ID_Empleado_Resp'
df_empleados_proyectos = df_empleados.join(
    df_proyectos,
    left_on="ID_Empleado",
    right_on="ID_Empleado_Resp",
    how="left" # Queremos todos los empleados y sus proyectos si los tienen
)
print("Join de empleados y proyectos con claves de nombres diferentes (left join):")
print(df_empleados_proyectos)
print("\n" + "="*50 + "\n")

## 4. **Join con múltiples claves:**
Pasa una lista de strings a on, `left_on`, o `right_on`.

```
# Ejemplo conceptual (necesitaríamos DFs adecuados)
df_A = pl.DataFrame({'k1': [1,1,2], 'k2': ['a','b','a'], 'valA': [10,20,30]})

df_B = pl.DataFrame({'k1': [1,1,2], 'k2': ['b','a','a'], 'valB': [100,200,300]})

df_multi_key_join = df_A.join(df_B, on=["k1", "k2"], how="inner")

print("\nJoin con múltiples claves ('k1', 'k2'):")
print(df_multi_key_join)
```



## 5. **Manejo de columnas con nombres duplicados (no claves) con suffix:**
Si después del `join`, hay columnas (que no son las claves de unión) con el mismo nombre provenientes de ambos DataFrames, Polars añadirá un sufijo al nombre de la columna del DataFrame *derecho*. Puedes personalizar este sufijo.

In [ ]:
df_deptos_con_nombre = df_departamentos.rename({"Nombre_Depto": "Nombre"}) # Renombramos para que coincida con 'Nombre' de empleados

# Ahora df_empleados y df_deptos_con_nombre tienen una columna 'Nombre'
df_join_sufijo = df_empleados.join(
    df_deptos_con_nombre,
    on="ID_Depto",
    how="left",
    suffix="_Depto" # Se añadirá a las columnas del DF derecho (df_deptos_con_nombre) si hay colisión
)
print("Join con sufijo para columnas duplicadas no clave:")
print(df_join_sufijo) # Verás 'Nombre' (del empleado) y 'Nombre_Depto' (del departamento)
print("\n" + "="*50 + "\n")

## **Breve Mención de `join_asof`**

Polars también ofrece `join_asof` para uniones "a partir de" o "más cercanas".

Es muy útil en series temporales para unir observaciones basándose en la marca de tiempo más cercana (pero no posterior, o dentro de una tolerancia).


```
# Ejemplo conceptual:
df_trades = pl.DataFrame({
     "time": [datetime(2025,1,1,9,0,0), datetime(2025,1,1,9,0,5), datetime(2025,1,1,9,0,12)],
     "price": [100, 101, 100.5], "stock": ["A", "A", "A"]
  })

df_quotes = pl.DataFrame({
     "time": [datetime(2025,1,1,9,0,0), datetime(2025,1,1,9,0,4), datetime(2025,1,1,9,0,10), datetime(2025,1,1,9,0,11)],
     "bid": [99.9, 99.8, 100.4, 100.3], "ask": [100.1, 100.0, 100.6, 100.5], "stock": ["A", "A", "A", "A"]
  })

# Unir cada trade con la cotización más reciente ANTES o EN el momento del trade.
df_trades_con_quotes = df_trades.join_asof(df_quotes, on="time", by="stock", strategy="backward")

print(df_trades_con_quotes)
```



---
# **Entrada y Salida de Datos (I/O).**

Saber cómo cargar datos eficientemente en Polars y cómo guardar tus resultados es tan crucial como la manipulación misma.

Polars destaca en esto, especialmente con su API scan_* para trabajar con datasets más grandes que la memoria RAM (modo `lazy`) y su excelente soporte para formatos columnares como `Parquet`.

## 1. **Leer Datos desde Archivos**

Las funciones de lectura en Polars generalmente comienzan con pl.read_* (para modo eager, carga todo en memoria) o pl.scan_* (para modo lazy, crea un plan).

### **Leer Archivos CSV**

  * `pl.read_csv()` (Modo `Eager`): Carga el CSV completo en un DataFrame en memoria.

In [ ]:
import polars as pl
import io # Para simular archivos

# Simular contenido de un archivo CSV
csv_data_polars = """id,nombre,valor,categoria
1,Manzana,1.2,Fruta
2,Leche,0.9,Lácteo
3,Pan,2.0,Panadería
4,Naranja,N/A,Fruta
"""
# En un caso real: df_eager_csv = pl.read_csv("ruta/a/archivo.csv")

df_eager_csv = pl.read_csv(
    source=io.StringIO(csv_data_polars), # Usamos StringIO para leer la cadena
    has_header=True,
    separator=',',
    null_values="N/A", # Definir qué strings son nulos
    dtypes={"valor": pl.Float32, "id": pl.Int16} # Especificar tipos
)
print("--- Leer CSV (Modo Eager) ---")
print("DataFrame desde read_csv:")
print(df_eager_csv)
print(df_eager_csv.schema)
print("\n" + "="*50 + "\n")

  * `pl.scan_csv()` (**Modo Lazy - ¡Muy Recomendado para Archivos Grandes!**):

No carga los datos inmediatamente. En su lugar, escanea el archivo para inferir el schema y crea un LazyFrame (un plan de cómo cargar y procesar los datos). Las operaciones se ejecutan solo cuando llamas a `.collect()`.

In [ ]:
# En un caso real: lf_lazy_csv = pl.scan_csv("ruta/a/archivo_grande.csv")
lf_lazy_csv = pl.scan_csv(
    source=io.StringIO(csv_data_polars), # Usamos la misma cadena
    has_header=True,
    separator=',',
    null_values="N/A",
    # dtypes se puede especificar, o Polars intentará inferirlo (con with_infer_schema_length)
    infer_schema_length=100 # Cuántas filas escanear para inferir el schema
)
print("--- Leer CSV (Modo Lazy con scan_csv) ---")
print("Tipo de objeto después de scan_csv:", type(lf_lazy_csv))
# <class 'polars.lazyframe.frame.LazyFrame'>

# Construyes tu consulta en el LazyFrame
query_lazy = (
    lf_lazy_csv
    .filter(pl.col("valor") > 1.0)
    .select(["nombre", "categoria", pl.col("valor") * 2 .alias("valor_duplicado")])
    .group_by("categoria")
    .agg(pl.col("valor_duplicado").sum())
)
print("\nPlan de consulta (LazyFrame):")
print(query_lazy) # Muestra el plan lógico

# Ejecutas la consulta y obtienes el DataFrame resultante con .collect()
df_resultado_lazy = query_lazy.collect()
print("\nDataFrame resultado después de .collect():")
print(df_resultado_lazy)
print("\n" + "="*50 + "\n")

### **Beneficios de scan_csv (y otros `scan_`*):**


* Optimización de Consultas: Polars puede optimizar toda la cadena de operaciones antes de tocar los datos.
* Out-of-Core: Puede procesar archivos más grandes que tu RAM porque solo carga en memoria los trozos (chunks) que necesita en cada momento.
* Proyección de Columnas (Predicate Pushdown): Si solo seleccionas ciertas columnas o filtras filas, Polars intentará leer solo los datos necesarios del disco.

## 2. **Leer Archivos Excel con `pl.read_excel()`:**
Polars usa motores como calamine (escrito en Rust, a menudo el predeterminado y muy rápido) o xlsx2csv para leer archivos Excel.
Asegúrate de tener el soporte para Excel instalado (`pip install polars[excel]`).

In [ ]:
# Simular un archivo Excel (creando un DF y guardándolo en un buffer de bytes)
df_para_excel_pl = pl.DataFrame({
    "Producto": ["X", "Y", "Z"],
    "Ventas_2025": [100, 150, 120],
    "Ventas_2024": [90, 140, 110]
})
buffer_excel = io.BytesIO()
# Polars no tiene un ExcelWriter tan complejo como Pandas para múltiples hojas directamente en write_excel.
# Para múltiples hojas, podrías escribir varios CSVs y luego unirlos, o usar una librería específica de Excel.
# Aquí escribimos una sola hoja.
df_para_excel_pl.write_excel(
    workbook=buffer_excel, # Puede ser una ruta o un buffer
    worksheet="Reporte_Ventas"
)
buffer_excel.seek(0) # Regresar el puntero para leer

# En un caso real: df_excel = pl.read_excel("ruta/a/archivo.xlsx", sheet_name="Hoja1")
df_excel_leido = pl.read_excel(
    source=buffer_excel,
    sheet_name="Reporte_Ventas", # También puede ser un índice (0, 1, ...)
    engine='calamine' # 'calamine' es rápido y no requiere MS Excel
)
print("--- Leer Excel ---")
print("DataFrame leído desde Excel simulado:")
print(df_excel_leido)
print("\n" + "="*50 + "\n")

## 3. **Leer Archivos Parquet con `pl.read_parquet()` y `pl.scan_parquet()`**
Parquet es un formato de archivo columnar altamente eficiente, optimizado para almacenamiento y análisis rápido.

Es el formato preferido para trabajar con Polars (y muchas otras herramientas de Big Data).

In [ ]:
# Simular un archivo Parquet
df_para_parquet = pl.DataFrame({
    "id": [1, 2, 3],
    "data": ["a", "b", "c"],
    "value": [10.1, 11.2, 12.3]
})
buffer_parquet = io.BytesIO()
df_para_parquet.write_parquet(buffer_parquet)
buffer_parquet.seek(0)

# Leer en modo eager
# En un caso real: df_parquet_eager = pl.read_parquet("ruta/a/archivo.parquet")
df_parquet_eager = pl.read_parquet(buffer_parquet)
print("--- Leer Parquet (Eager y Lazy) ---")
print("DataFrame desde read_parquet (eager):")
print(df_parquet_eager)

# Leer en modo lazy (MUY recomendado para Parquet)
buffer_parquet.seek(0) # Rebobinar el buffer para scan
# En un caso real: lf_parquet_lazy = pl.scan_parquet("ruta/a/archivo_grande.parquet")
lf_parquet_lazy = pl.scan_parquet(buffer_parquet)
print("\nLazyFrame desde scan_parquet:")
print(lf_parquet_lazy) # Muestra el plan

df_resultado_parquet_lazy = lf_parquet_lazy.filter(pl.col("value") > 11.0).collect()
print("\nResultado de scan_parquet después de filter y collect:")
print(df_resultado_parquet_lazy)
print("\n" + "="*50 + "\n")

## 4. **Mención de Otros Formatos de Lectura:**

* `pl.read_json()` / `pl.scan_ndjson()`: Para archivos JSON o Newline Delimited JSON.
* `pl.read_ipc()` / `pl.scan_ipc()`: Para formato Arrow IPC (Feather).
* Conectores para bases de datos (a menudo mediante librerías de terceros o adbc):
  * Polars puede leer desde bases de datos SQL, aunque la forma exacta puede depender del conector (`pip install polars[adbc]` `polars[connectorx]`).
  Por ejemplo, `pl.read_database(query="SELECT * FROM mi_tabla", connection=mi_conexion_uri)`.

## **Escribir Datos a Archivos:**

Los métodos para escribir DataFrames a archivos suelen ser `df.write_`*.

### 1. **Escribir a Archivos CSV con `df.write_csv()`:**

In [ ]:
df_para_escribir = pl.DataFrame({
    "colA": [1, 2, 3], "colB": ["x", "y", "z"]
})
print("--- Escribir a Archivos ---")
print("\nDataFrame para escribir:")
print(df_para_escribir)

buffer_escritura_csv_pl = io.StringIO()
df_para_escribir.write_csv(buffer_escritura_csv_pl, separator=";")
print("\nContenido CSV 'guardado' en buffer (separador ';'):")
print(buffer_escritura_csv_pl.getvalue())
# En un caso real: df_para_escribir.write_csv("salida.csv", separator=";")

### 2. **Escribir a Archivos Excel con `df.write_excel()`**
Requiere tener un motor como xlsxwriter instalado (`pip install xlsxwriter`).

In [ ]:
buffer_escritura_excel_pl = io.BytesIO()
try:
    df_para_escribir.write_excel(
        workbook=buffer_escritura_excel_pl,
        worksheet="MisDatosPolars",
        # Si quieres opciones de formato, la API es más limitada que la de Pandas
        # o podrías usar xlsxwriter directamente con los datos de Polars.
    )
    print("\n(Simulación) DataFrame 'guardado' en buffer de Excel.")
    # Para verificar: buffer_escritura_excel_pl.seek(0); df_leido = pl.read_excel(buffer_escritura_excel_pl)
except Exception as e:
    print(f"\nError escribiendo Excel (puede necesitar 'xlsxwriter'): {e}")
# En un caso real: df_para_escribir.write_excel("salida.xlsx", worksheet="HojaPrincipal")

### 3. **Escribir a Archivos Parquet con `df.write_parquet()`:**
Este es el método recomendado para guardar DataFrames de Polars si vas a volver a leerlos con Polars u otras herramientas compatibles con Arrow/Parquet, debido a su eficiencia.

In [ ]:
buffer_escritura_parquet_pl = io.BytesIO()
df_para_escribir.write_parquet(buffer_escritura_parquet_pl, compression="snappy") # snappy es una compresión rápida
print("\n(Simulación) DataFrame 'guardado' en buffer de Parquet con compresión snappy.")
# En un caso real: df_para_escribir.write_parquet("salida.parquet", compression="snappy")
print("\n" + "="*50 + "\n")

## **Interoperabilidad con Pandas, NumPy y PyArrow**

Polars está diseñado para coexistir y trabajar bien con otras bibliotecas del ecosistema PyData.

In [ ]:
print("--- Interoperabilidad ---")
# DataFrame de Polars de ejemplo
df_polars_interop = pl.DataFrame({"a": [1, 2], "b": [3.0, 4.5]})

# 1. A Pandas DataFrame
df_pandas_convertido = df_polars_interop.to_pandas()
print("\nConvertido a Pandas DataFrame:")
print(df_pandas_convertido)
print(type(df_pandas_convertido))

# 2. Desde un Pandas DataFrame
df_pandas_original = pd.DataFrame({"x": [10, 20], "y": ["hola", "mundo"]})
df_polars_desde_pandas = pl.from_pandas(df_pandas_original)
print("\nConvertido desde Pandas DataFrame:")
print(df_polars_desde_pandas)
print(type(df_polars_desde_pandas))

# 3. A NumPy array (puede ser una o varias columnas)
array_numpy = df_polars_interop.to_numpy() # Devuelve un array estructurado si hay múltiples tipos
print("\nConvertido a NumPy array (estructurado):")
print(array_numpy)
# Para una sola columna:
array_numpy_col_a = df_polars_interop["a"].to_numpy()
print("\nColumna 'a' como NumPy array:")
print(array_numpy_col_a)

# 4. A tabla PyArrow
tabla_arrow = df_polars_interop.to_arrow()
print("\nConvertido a tabla PyArrow:")
print(tabla_arrow)
print(type(tabla_arrow))

# 5. Desde tabla PyArrow
df_polars_desde_arrow = pl.from_arrow(tabla_arrow)
print("\nConvertido desde tabla PyArrow:")
print(df_polars_desde_arrow)